Prediction of rain in Australia using random forest model. Dataset from Kaggle:
https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package


In [44]:
import pandas as pd 
import numpy as np
import time


rain = pd.read_csv("/home/kata/zajecia/repo/jdszr14-materialy-ml/random_forest/data/weatherAUS.csv")

In [ ]:
rain.head()

In [2]:
#checking the shape of df
shape = rain.shape
print(f"The shape of the dataset is: {shape}.")

The shape of the dataset is: (142193, 24).


In [3]:
#checking missing values in df

rain.isnull().sum().sort_values()

Date                 0
RISK_MM              0
RainTomorrow         0
Location             0
MaxTemp            322
MinTemp            637
Temp9am            904
WindSpeed9am      1348
Rainfall          1406
RainToday         1406
Humidity9am       1774
WindSpeed3pm      2630
Temp3pm           2726
Humidity3pm       3610
WindDir3pm        3778
WindGustSpeed     9270
WindGustDir       9330
WindDir9am       10013
Pressure3pm      13981
Pressure9am      14014
Cloud9am         53657
Cloud3pm         57094
Evaporation      60843
Sunshine         67816
dtype: int64

In [4]:
def drop_columns(df, thr = 0.5):
    missing_percentages = df.isna().mean()
    col_to_drop = missing_percentages[missing_percentages > thr].index
    new_df=df.drop(columns = col_to_drop)
    return new_df, list(col_to_drop)

In [18]:
dropped_col_df, dropped_columns = drop_columns(rain, 0.35)


In [19]:
#listing the cdropped columns
dropped_columns

['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm']

In [20]:
dropped_col_df.isna().mean()

Date             0.000000
Location         0.000000
MinTemp          0.004480
MaxTemp          0.002265
Rainfall         0.009888
WindGustDir      0.065615
WindGustSpeed    0.065193
WindDir9am       0.070418
WindDir3pm       0.026570
WindSpeed9am     0.009480
WindSpeed3pm     0.018496
Humidity9am      0.012476
Humidity3pm      0.025388
Pressure9am      0.098556
Pressure3pm      0.098324
Temp9am          0.006358
Temp3pm          0.019171
RainToday        0.009888
RISK_MM          0.000000
RainTomorrow     0.000000
dtype: float64

In [21]:
dropped_col_df.drop(columns = ["Date", "Location","RISK_MM"], axis =1, inplace=True)

In [22]:
#removing records with missing values
dropped_col_df=dropped_col_df.dropna(how = 'any')

In [23]:
dropped_col_df

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No
1,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No
2,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No
3,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No
4,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,3.5,21.8,0.0,E,31.0,ESE,E,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9,No,No
142189,2.8,23.4,0.0,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4,No,No
142190,3.6,25.3,0.0,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5,No,No
142191,5.4,26.9,0.0,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1,No,No


In [24]:

dropped_col_df['RainToday'].replace({'No': 0, 'Yes': 1}, inplace = True)
dropped_col_df['RainTomorrow'].replace({'No': 0, 'Yes': 1}, inplace = True)

In [28]:
# using get_dummies method to encode categorical data

categorical_features = ["WindGustDir", "WindDir9am", "WindDir3pm"]


for col in categorical_features:
    print(np.unique(dropped_col_df[col]))

dropped_col_df = pd.get_dummies(dropped_col_df, columns=categorical_features)



['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']


In [29]:
dropped_col_df.replace({'False': 0, "True":1}, inplace=True)

In [30]:
dropped_col_df

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,False,False,False,False,False,False,False,False,True,False
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,False,False,False,False,False,False,False,False,False,True
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,False,False,False,False,False,False,False,False,False,True
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,False,False,False,False,False,False,False,False,False,False
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,3.5,21.8,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,1021.2,...,False,False,False,False,False,False,False,False,False,False
142189,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,False,False,False,False,False,False,False,False,False,False
142190,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,False,False,False,False,False,False,False,False,False,False
142191,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,False,False,False,False,False,False,False,False,True,False


In [32]:
# scaling data using MinMaxScaler

from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
scaler.fit(dropped_col_df)
dropped_col_df = pd.DataFrame(scaler.transform(dropped_col_df), index =dropped_col_df.index, columns = dropped_col_df.columns)

dropped_col_df.iloc[0:5]

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.513064,0.446154,0.001632,0.289062,0.211765,0.258824,0.71,0.22,0.449587,0.4800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.370546,0.494505,0.000000,0.289062,0.023529,0.235294,0.44,0.25,0.497521,0.4912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.501188,0.507692,0.000000,0.304688,0.200000,0.282353,0.38,0.30,0.447934,0.5056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.413302,0.558242,0.000000,0.132812,0.105882,0.082353,0.45,0.16,0.613223,0.5712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.610451,0.652747,0.002720,0.265625,0.058824,0.211765,0.82,0.33,0.500826,0.4624,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#preparing dataset 

y = dropped_col_df["RainTomorrow"]
X = dropped_col_df.drop(columns="RainTomorrow")


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [50]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

t_0 = time.time()

log_reg = LogisticRegression(max_iter=1000, random_state=0)
log_reg.fit(X_train,y_train)

y_pred_log = log_reg.predict(X_test)

log_reg_acc = accuracy_score(y_test, y_pred_log)

t_1 = time.time() - t_0

print(f"Accuracy for a testing dataset for logistic regression model is: {log_reg_acc}. Time taken: {t_1}.")

Accuracy on a testing dataset for logistic regression is: 0.849636932522581. Time taken: 2.8303589820861816.


In [51]:
#Decision tree 

from sklearn.tree import DecisionTreeClassifier


t_0 = time.time()

dec_tree = DecisionTreeClassifier(max_depth=6, random_state=0)
dec_tree.fit(X_train,y_train)

y_pred_tree = log_reg.predict(X_test)

dec_tree_acc = accuracy_score(y_test, y_pred_tree)

t_1 = time.time() - t_0

print(f"Accuracy for a testing dataset for decision tree model is: {dec_tree_acc}. Time taken: {t_1}.")

Accuracy for a testing dataset for decision tree model is: 0.849636932522581. Time taken: 0.5496816635131836.


In [53]:
# Random forest

from sklearn.ensemble import RandomForestClassifier

t_0 = time.time()

r_forest = RandomForestClassifier(n_estimators = 1000, max_depth=6, random_state=0)
r_forest.fit(X_train,y_train)

y_pred_forest = r_forest.predict(X_test)

r_forest_acc = accuracy_score(y_test, y_pred_forest)

t_1 = time.time() - t_0

print(f"Accuracy for a testing dataset for random forest model is: {r_forest_acc}. Time taken: {t_1}.")


Accuracy for a testing dataset for random forest model is: 0.837298541826554. Time taken: 57.10147023200989.
